Vamos a compilar y ejecutar el siguiente ejemplo:

    gcc carretero-ejemplo-3.14-6.c -lpthread    

In [3]:
//%cflags:-lpthread

#include <pthread.h>
#include <stdio.h>

void* func(void* arg) {
  printf("Thread %lu\n", pthread_self());
  pthread_exit(0);
}

int main(void) {
  pthread_t th1, th2;
  
  pthread_create(&th1, NULL, func, NULL);
  pthread_create(&th2, NULL, func, NULL);
  
  printf("El thread principal continúa\n");

  pthread_join(th1, NULL);
  pthread_join(th2, NULL);
  
  return 0;
}

El thread principal continúa
Thread 140325716932352
Thread 140325725325056


En el siguiente ejemplo hacemos uso de los argumentos:

    gcc thread1.c -lpthread

In [4]:
//%cflags:-lpthread

#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <string.h>
#include <pthread.h>

void *thread_function(void *arg);
char message[] = "Hello World";

int main() {
  int res;
  pthread_t a_thread;
  void *thread_result;
  res = pthread_create(&a_thread, NULL, thread_function, (void *)message);
  if (res != 0) {
    perror("Thread creation failed");
    exit(EXIT_FAILURE);
  }
  printf("Waiting for thread to finish...\n");
  res = pthread_join(a_thread, &thread_result);
  if (res != 0) {
    perror("Thread join failed");
    exit(EXIT_FAILURE);
  }
  printf("Thread joined, it returned %s\n", (char *)thread_result);
  printf("Message is now %s\n", message);
  exit(EXIT_SUCCESS);
}

void *thread_function(void *arg) {
  printf("Thread_function is running. Argument was %s\n", (char *)arg);
  sleep(3);
  strcpy(message, "Bye!");
  pthread_exit("Thank you for the CPU time");
}


Waiting for thread to finish...
Thread_function is running. Argument was Hello World
Thread joined, it returned Thank you for the CPU time
Message is now Bye!


En el siguiente ejemplo veremos cómo realizar una ejecución simultánea de dos hebras, lo que nos llevará de forma automática a la sincronización en el módulo 3.3

In [5]:
//%cflags:-lpthread

#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <pthread.h>

// SEVERE: Por simiplicidad, omitimos todas las comprobaciones de error.
void *thread_function(void *arg);
int run_now = 1;
char message[] = "Hello World";

int main() {
  int res;
  pthread_t a_thread;
  void *thread_result;
  int print_count1 = 0;
  
  res = pthread_create(&a_thread, NULL, thread_function, (void *)message);
  
  while(print_count1++ < 20) {
    if (run_now == 1) {
      printf("1");
      fflush(stdout);
      run_now = 2;
    }
    else {
      sleep(1);
    }
  }
  
  printf("\nWaiting for thread to finish...\n");
  res = pthread_join(a_thread, &thread_result);
  printf("Thread joined\n");
  exit(EXIT_SUCCESS);
}

void *thread_function(void *arg) {
  int print_count2 = 0;
  
  while(print_count2++ < 20) {
    if (run_now == 2) {
      printf("2");
      fflush(stdout);
      run_now = 1;
    }
    else {
      sleep(1);
    }
  }
  sleep(3);
  pthread_exit(0);
}

12121212121212121212
Waiting for thread to finish...
Thread joined
